Github Analysis!

In [ ]:
from spark_utils import quick_start
spark = quick_start("TestConnection")

In [ ]:
# Load your data
df = spark.read.format("delta").load("s3a://delta-lake/gold/gold_github_technology_daily_activity")


In [ ]:
df.printSchema()

In [ ]:
df.show(5, truncate=False)

In [ ]:
df.groupBy("technology") \
  .sum("daily_mentions") \
  .orderBy("sum(daily_mentions)", ascending=False) \
  .show(10, truncate=False)

In [ ]:
df.filter(df.technology != "github") \
  .select("technology", "data_quality_score", "source_record_count") \
  .orderBy("data_quality_score", ascending=False) \
  .show(20, truncate=False)

In [ ]:
df.filter((df.technology != "github") & (df.data_quality_score > 50)) \
  .orderBy(df.daily_mentions.desc()) \
  .select("technology", "daily_mentions", "data_quality_score", "primary_repository") \
  .show(20, truncate=False)

In [ ]:
df.filter((df.daily_mentions > 5000) & (df.data_quality_score > 51)) \
  .orderBy(df.daily_mentions.desc()) \
  .select("technology", "daily_mentions", "data_quality_score", "primary_repository") \
  .show(truncate=False)

In [ ]:
# Quick Duplicate Check in Jupyter
print("🔍 CHECKING FOR DUPLICATES IN SILVER AND GOLD")
print("=" * 50)

# 1. SILVER TABLE DUPLICATE CHECK
print("\n🥈 SILVER TABLE ANALYSIS:")
print(f"Total Silver records: {dfsilver.count():,}")

# Check for duplicate (date, technology) combinations
silver_duplicates = dfsilver.groupBy("date", "technology").count().filter("count > 1")
duplicate_count = silver_duplicates.count()

print(f"Duplicate (date, technology) combinations: {duplicate_count}")

if duplicate_count > 0:
    print("\n❌ SILVER HAS DUPLICATES!")
    print("Sample duplicates:")
    silver_duplicates.orderBy("count", ascending=False).show(10)
    
    # Show total duplicate records
    total_duplicates = silver_duplicates.agg({"count": "sum"}).collect()[0][0] - duplicate_count
    print(f"Total duplicate records: {total_duplicates}")
else:
    print("✅ No duplicates found in Silver")

# 2. LOAD GOLD TABLE AND CHECK
print("\n🥇 GOLD TABLE ANALYSIS:")
try:
    dfgold = spark.read.format("delta").load("s3a://delta-lake/gold/gold_github_technology_daily_activity")
    print(f"Total Gold records: {dfgold.count():,}")
    
    # Check for duplicate (date, technology) combinations  
    gold_duplicates = dfgold.groupBy("date", "technology").count().filter("count > 1")
    gold_duplicate_count = gold_duplicates.count()
    
    print(f"Duplicate (date, technology) combinations: {gold_duplicate_count}")
    
    if gold_duplicate_count > 0:
        print("\n❌ GOLD HAS DUPLICATES!")
        print("Sample duplicates:")
        gold_duplicates.orderBy("count", ascending=False).show(10)
        
        # Show total duplicate records
        total_gold_duplicates = gold_duplicates.agg({"count": "sum"}).collect()[0][0] - gold_duplicate_count
        print(f"Total duplicate records: {total_gold_duplicates}")
    else:
        print("✅ No duplicates found in Gold")
        
except Exception as e:
    print(f"Could not load Gold table: {e}")

# 3. RECORD COUNT BY DATE (to spot anomalies)
print("\n📊 RECORD COUNTS BY DATE:")
print("\nSilver records per date:")
dfsilver.groupBy("date").count().orderBy("date").show(20)

if 'dfgold' in locals():
    print("\nGold records per date:")
    dfgold.groupBy("date").count().orderBy("date").show(20)

# 4. QUICK STATS
print("\n📈 QUICK STATS:")
print("Silver date range:", dfsilver.agg({"date": "min"}).collect()[0][0], "to", dfsilver.agg({"date": "max"}).collect()[0][0])
print("Silver unique dates:", dfsilver.select("date").distinct().count())
print("Silver unique technologies:", dfsilver.select("technology").distinct().count())

if 'dfgold' in locals():
    print("Gold date range:", dfgold.agg({"date": "min"}).collect()[0][0], "to", dfgold.agg({"date": "max"}).collect()[0][0])
    print("Gold unique dates:", dfgold.select("date").distinct().count())
    print("Gold unique technologies:", dfgold.select("technology").distinct().count())